In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import cv2
from scipy.linalg import svd
from glob import glob


In [3]:
import pandas as pd

In [4]:
submission= pd.read_csv('/kaggle/input/image-matching-challenge-2023/sample_submission.csv')

In [5]:
import cv2
import numpy as np
rotation_matrix_column=[]
translation_vector_column=[]
def sfm(images,submission,j):
    # Step 1: Feature extraction and matching
    sift = cv2.SIFT_create()
    keypoints = []
    descriptors = []

    for gray in images:
        kp, desc = sift.detectAndCompute(gray, None)
        keypoints.append(kp)
        descriptors.append(desc)

    # Step 2: Feature matching
    matcher = cv2.BFMatcher()
    matches = []

    for i in range(len(descriptors) - 1):
        matches.append(matcher.knnMatch(descriptors[i], descriptors[i+1], k=2))

    # Step 3: Camera pose estimation
    camera_matrix = np.eye(3)
    poses = [np.zeros((3, 1))]

    for i in range(len(matches)):
        src_pts = np.float32([keypoints[i][m[0].queryIdx].pt for m in matches[i]]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints[i+1][m[0].trainIdx].pt for m in matches[i]]).reshape(-1, 1, 2)

        essential_matrix, _ = cv2.findEssentialMat(src_pts, dst_pts, camera_matrix)
        _, rotation_matrix, translation_vector, _ = cv2.recoverPose(essential_matrix, src_pts, dst_pts, camera_matrix)
        rotation_matrix_column.append(rotation_matrix)
        translation_vector_column.append(translation_vector)
        poses.append(poses[i] + rotation_matrix.dot(translation_vector))
        # print(rotation_matrix)
        string1=""
        string2=""
        for row in rotation_matrix:
          string1=string1+';'.join(row.astype(str))
        for row in translation_vector:
          string2=string2+';'.join(row.astype(str))+";"
        string2=string2[:-1]
        newrow=pd.DataFrame({'dataset':[dataset],'scene':[scene],'image_path':[image_path[j]],'rotation_matrix':[string1],'translation_vector':[string2]})
        submission=pd.concat([submission,newrow], ignore_index=True)
        j=j+1
        # submission['rotation_matrix']=submission['rotation_matrix'].append(pd.Series([string1]), ignore_index=True)

        # submission['translation_vector']=submission['translation_vector'].append(pd.Series([string2]), ignore_index=True)
    # Step 4: Triangulation
    points_3d = []

    for i in range(len(poses) - 1):
        src_pts = np.float32([keypoints[i][m[0].queryIdx].pt for m in matches[i]]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints[i+1][m[0].trainIdx].pt for m in matches[i]]).reshape(-1, 1, 2)

        projection_matrix_1 = np.hstack((np.eye(3), np.zeros((3, 1))))
        projection_matrix_2 = np.hstack((rotation_matrix, translation_vector))

        points = cv2.triangulatePoints(projection_matrix_1, projection_matrix_2, src_pts, dst_pts)
        points /= points[3]
        points_3d.append(points)
#     submission.to_csv("submission.csv",index=False)
#    submission.to_csv("submission.csv",mode='a', header=False, index=False)
    return poses, points_3d


In [6]:
submission.to_csv("submission.csv",index=False)

In [7]:
dataset = '2cfa01ab573141e4'
scene = '2fa124afd1f74f38'
src = f'/kaggle/input/image-matching-challenge-2023/train/heritage/dioscuri'
image_path=glob(f'{src}/images/*')
images = [cv2.cvtColor(cv2.imread(im), cv2.COLOR_BGR2GRAY) for im in glob(f'{src}/images/*')[:3]]
str1='0;0;0;0;0;0;0;0;0'
str2='0;0;0'
nrow=pd.DataFrame({'dataset':[dataset],'image_path':[image_path[0]],'scene':[scene],'rotation_matrix':[str1],'translation_vector':[str2]})
submission=pd.concat([submission,nrow], ignore_index=True)

j=1
poses, points_3d = sfm(images,submission,j)


In [8]:

submission

,image_path,dataset,scene,rotation_matrix,translation_vector
0,2cfa01ab573141e4/2fa124afd1f74f38/images/09387...,2cfa01ab573141e4,2fa124afd1f74f38,0.3700584878256019;0.7870891328233803;0.632781...,0.12296637725515835;0.6206590405309732;0.10571...
1,2cfa01ab573141e4/2fa124afd1f74f38/images/11c07...,2cfa01ab573141e4,2fa124afd1f74f38,0.24107462449034978;0.6324667958150785;0.66208...,0.30084987667667396;0.6212308657300727;0.66494...
2,2cfa01ab573141e4/2fa124afd1f74f38/images/11d54...,2cfa01ab573141e4,2fa124afd1f74f38,0.16107224414906363;0.827391324245376;0.044308...,0.293163905050011;0.9682221798154219;0.8272006...
3,/kaggle/input/image-matching-challenge-2023/tr...,2cfa01ab573141e4,2fa124afd1f74f38,0;0;0;0;0;0;0;0;0,0;0;0
